In [133]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,575.0000,3.125000,3.481845e-03,SHA-256,2.969952,150.260870,0.010881,1.634953,0.550498
BTC,663.0000,3.161729,1.000000e+00,SHA-256,739.584757,130.316742,3.161729,412.026282,0.557105
DOGE,63.0000,10000.000000,2.719109e-06,Scrypt,0.002840,1371.428571,0.027191,37.290643,13131.267403
KAS,0.9991,65.461431,1.103704e-06,kHeavyHash,1.389796,86477.830047,0.000072,6.248028,4.495644
LTC,144.0000,6.250000,1.319000e-03,Scrypt,0.002275,600.000000,0.008244,4.946250,2174.180864
RVN,59.0000,2500.000000,1.500000e-07,KawPow,0.000008,1464.406780,0.000375,0.549153,69368.617053


In [134]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,daily_consumption_btc
SCRYPT,1.334241e-04,15582.604217,2.079094
SHA256ASICBOOST,7.034401e+00,0.540240,3.800262
KAWPOW,7.834915e-07,81332.293309,0.063723
KHEAVYHASH,5.543144e-02,4.481493,0.248416


In [135]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_dollar'] = round(nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate, 2)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_dollar
SCRYPT,1.014023e-04,1.580112,0.065838,6466.86
SHA256ASICBOOST,5.346145e+00,2.888199,0.120342,11820.41
KAWPOW,5.954536e-07,0.048430,0.002018,198.21
KHEAVYHASH,4.212789e-02,0.188796,0.007866,772.68


In [136]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped
BCH,SHA-256,2.969952,5.346145e+00,1.800078,4.454928e-01
BTC,SHA-256,739.584757,5.346145e+00,0.007229,5.346145e+00
DOGE,Scrypt,0.002840,1.014023e-04,0.035707,1.014023e-04
KAS,kHeavyHash,1.389796,4.212789e-02,0.030312,4.212789e-02
LTC,Scrypt,0.002275,1.014023e-04,0.044573,1.014023e-04
RVN,KawPow,0.000008,5.954536e-07,0.075217,5.954536e-07


In [138]:
min_order_amount_btc = 0.0001 # BTC
order_duration = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['speed_h'] = min_order_amount_btc / (nh_market_df['paying_btc_eh_day'] * min_order_amount_btc)
speed_df

,speed_h
SCRYPT,0.000064
SHA256ASICBOOST,1.851031
KAWPOW,0.000012
KHEAVYHASH,0.223140
